In [2]:
import sqlite3
import pymongo
import pandas as pd
import requests

In [60]:
def pullmatchdata(season):
    conn = sqlite3.connect("database.sqlite")
    cur = conn.cursor()
    search_home = f"""SELECT HomeTeam, Season, Div, SUM(FTHG) AS HomeGoals, COUNT(FTR) AS HomeWins 
                      FROM matches
                      WHERE Season = {season} AND FTR = 'H' AND (Div = 'D1' OR Div = 'E0')
                      GROUP BY HomeTeam;"""
    dfhome = pd.read_sql_query(search_home, conn)
    search_away = f"""SELECT AwayTeam, SUM(FTAG) AS AwayGoals, COUNT(FTR) AS AwayWins 
                      FROM matches
                      WHERE Season = {season} AND FTR = 'A' AND (Div = 'D1' OR Div = 'E0')
                      GROUP BY AwayTeam;"""
    dfaway = pd.read_sql_query(search_away, conn)
    df = pd.merge(dfhome, dfaway, how='inner', left_on='HomeTeam', right_on='AwayTeam')
    del df['AwayTeam']
    df.rename(columns = {'HomeTeam':'TeamName'}, inplace = True)
    df['TotalGoals'] = df['HomeGoals'] + df['AwayGoals']
    df['TotalWins'] = df['HomeWins'] + df['AwayWins']
    return df